# LIME
- (Local Interpretable Model-agnostic Explanation)
- model agnostic ( = 모델의 종류와 '상관 없이'! )

## 1. Import Dataset

In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_validate

from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
from matplotlib import font_manager, rc, rcParams

def set_korea_font():
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
    rcParams.update({'font.size': 11})
    rcParams['axes.unicode_minus'] = False  
set_korea_font()

In [9]:
# main data
house = pd.read_csv('kc_house_data.csv')

# additional data
dum = pd.read_csv('dummy.csv').iloc[:,2:]
dum.columns = ['밸뷰(배드타운)','메디나(부촌)','타코마 및 교외']

# new data
house = pd.merge(house,dum,left_index=True,right_index=True)

In [10]:
print(house.shape)
house.head()

(21613, 24)


id             date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900.0         3       1.00         1180   
1  6414100192  20141209T000000  538000.0         3       2.25         2570   
2  5631500400  20150225T000000  180000.0         2       1.00          770   
3  2487200875  20141209T000000  604000.0         4       3.00         1960   
4  1954400510  20150218T000000  510000.0         3       2.00         1680   

   sqft_lot  floors  waterfront  view    ...     yr_built  yr_renovated  \
0      5650     1.0           0     0    ...         1955             0   
1      7242     2.0           0     0    ...         1951          1991   
2     10000     1.0           0     0    ...         1933             0   
3      5000     1.0           0     0    ...         1965             0   
4      8080     1.0           0     0    ...         1987             0   

   zipcode      lat     long  sqft_living15  sqft_lot15  밸뷰(배드타운)  메디나(부촌)  \
0    98178  47.5112 -122.257           1340        5650         0        0   
1    98125  47.7210 -122.319           1690        7639         0        0   
2    98028  47.7379 -122.233           2720        8062         0        0   
3    98136  47.5208 -122.393           1360        5000         0        0   
4    98074  47.6168 -122.045           1800        7503         1        0   

   타코마 및 교외  
0         1  
1         0  
2         0  
3         0  
4         0  

[5 rows x 24 columns]

### ( train/test split & standard scaling )

In [64]:
col = house.columns.difference(['price','date','id','lat','long','zipcode'])

X = house[col]
y = house['price']

In [65]:
# Scaling
sc = StandardScaler()
X_scaled = pd.DataFrame(sc.fit_transform(X), columns = X.columns)
X_scaled.head()

bathrooms  bedrooms  condition    floors     grade  sqft_above  \
0  -1.447464 -0.398737  -0.629187 -0.915427 -0.558836   -0.734708   
1   0.175607 -0.398737  -0.629187  0.936506 -0.558836    0.460841   
2  -1.447464 -1.473959  -0.629187 -0.915427 -1.409587   -1.229834   
3   1.149449  0.676485   2.444294 -0.915427 -0.558836   -0.891699   
4  -0.149007 -0.398737  -0.629187 -0.915427  0.291916   -0.130895   

   sqft_basement  sqft_living  sqft_living15  sqft_lot  sqft_lot15      view  \
0      -0.658681    -0.979835      -0.943355 -0.228321   -0.260715 -0.305759   
1       0.245141     0.533634      -0.432686 -0.189885   -0.187868 -0.305759   
2      -0.658681    -1.426254       1.070140 -0.123298   -0.172375 -0.305759   
3       1.397515    -0.130550      -0.914174 -0.244014   -0.284522 -0.305759   
4      -0.658681    -0.435422      -0.272190 -0.169653   -0.192849 -0.305759   

   waterfront  yr_built  yr_renovated   메디나(부촌)  밸뷰(배드타운)  타코마 및 교외  
0   -0.087173 -0.544898     -0.210128 -0.210615 -0.559947  1.702483  
1   -0.087173 -0.681079      4.746678 -0.210615 -0.559947 -0.587377  
2   -0.087173 -1.293892     -0.210128 -0.210615 -0.559947 -0.587377  
3   -0.087173 -0.204446     -0.210128 -0.210615 -0.559947 -0.587377  
4   -0.087173  0.544548     -0.210128 -0.210615  1.785883 -0.587377

In [66]:
# Train/Test split
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

## 2. Model : AdaBoost
- LIME은 XGBoost에서 불가능!

In [130]:
ada = AdaBoostRegressor(random_state=42)
ada.fit(train_x, train_y)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=42)

In [131]:
from sklearn.metrics import mean_squared_error as mse
from math import sqrt

ada_predict = ada.predict(test_x)
print("RMSE : {}".format(sqrt(mse(ada_predict, test_y))))

RMSE : 358081.3103979877


## 3. LIME

In [33]:
import lime
import lime.lime_tabular as lt

In [134]:
# X_scaled 말고, X를 써서 cat_features 찾아내자 우선!
categorical_features = np.argwhere(np.array([len(set(X.values[:,x])) for x in range(X.shape[1])]) <= 10).flatten()
categorical_features

array([ 2,  3, 11, 12, 15, 16, 17], dtype=int64)

In [135]:
explainer = lt.LimeTabularExplainer(training_data=X_scaled.values, mode='regression',
                                                   feature_names=X_scaled.columns, categorical_features=categorical_features, verbose=True)

In [138]:
# 밑에 ada에는, 위에서 찾은 최적의 hyperparameter를 대입!
# (주의) fitting 시킬 때, train set로 하지 말고, train+test 합친 것으로!

dt = DecisionTreeRegressor()
ada = AdaBoostRegressor(dt,learning_rate=1.4,n_estimators=30,random_state=1)

ada.fit(X_scaled,y/1000)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                       max_depth=None,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       presort=False,
                                                       random_state=None,
                                                       splitter='best'),
                  learning_rate=1.4, loss='linear', n_estimators=30,
                  rando

In [141]:
# 여기서 test하고 싶은 data는 모든 것 다 가능! ( train이든 test 이든 )
# ex) 73번째 data 설명해봐!
exp = explainer.explain_instance(X_scaled.values[73], ada.predict, num_features=5)
exp.show_in_notebook(show_table=True)

Intercept 876.8838316253812
Prediction_local [442.68590829]
Right: 360.0
